# Snow Depth Estimates with ICESat-2

This notebook uses a combination of ICESat-2 and airborne lidar to derive snow depth. It uses data from the SnowEx 2023 campaign as an example, but can be applied to other locations if a shapefile or geoJSON is given.

This notebook is adapted from the 2023 ICESat-2 Hackweek, originally developed by Zachary Fair and Karina Zikan.

## User input

Acceptable field site IDs over Alaska are:
* `cffl`: Creamer's Field/Farmer's Loop
* `cpcrw`: Caribou/Poker Creek Experimental Watershed
* `bcef`: Bonanza Creek Experimental Forest
* `acp`: Arctic Coastal Plain
* `utk`: Toolik Research Station

Acceptable IDs for Sliderule ATL08 class (use numeric ID):
* `No classification:` -1
* `atl08_unclassified`: 0
* `atl08_noise`: 1
* `atl08_canopy`: 2
* `atl08_top_of_canopy`: 3
* `atl08_ground`: 4

In [31]:
# Field site ID
field_id = '/home/jovyan/shared-public/SnowPit/snow-depth/toolik_lidar_boxes.geojson'

# Snow-on (True) or snow-off (False) analysis
snow_on = True

# Use March UAF data ('mar') or October depths ('oct')
uaf_depths = 'mar'

# Base data path
path = '/home/jovyan/icesat2-snowex'

# Desired RGT and date range for data queries. Set rgt to "all" if
# all ground tracks are desired
date_range = ['2023-03-01', '2023-04-01']
rgt = '152'

# SlideRule parameters (optional)
cnf_surface = 4
atl08_class = 4
segment_length = 40
res = 20

A breakdown of the SlideRule parameters above:

`cnf_surface`: The confidence level of the ICESat-2 photons.
* High-confidence photons (recommended for snow): 4
* High-/medium-confidence photons: 3
* High-/medium-/low-confidence photons: 2
* Signal photons (high/medium/low) and noise: 1
* Signal photons, noise, and solar background (not recommended): 0

`segment_length`: The along-track length to sample and aggregate photons, in meters. Currently set at 40 m, the resolution of the ATL06 product.

`res`: The along-track resolution of the returned data product. Currently set at 20 m to match ATL06.

## Read ICESat-2 data
To load the ICESat-2 data with minimal effort from the user, we will use SlideRule in the below function.

In [27]:
from sliderule import sliderule, icesat2

def atl06srq(field_geojson, date_range, rgt, cnf_surface, atl08_class, 
             segment_length, res):
    # Initiate SlideRule
    icesat2.init('slideruleearth.io', verbose=False)

    # Load geoJSON for site of interest
    region = sliderule.toregion(field_geojson)['poly']

    # Convert user-defined ATL08 class ID to string readable by SlideRule
    atl08_ids = {-1: 'None',
                 0: 'atl08_unclassified',
                 1: 'atl08_noise',
                 2: 'atl08_canopy',
                 3: 'atl08_top_of_canopy',
                 4: 'atl08_ground'}

    # Construct dictionary of parameters
    time_root = 'T00:00:00Z'
    parms = {
             "poly": region,
             "srt": icesat2.SRT_LAND,
             "cnf": cnf_surface,
             "len": segment_length,
             "res": res,
             "t0": date_range[0]+time_root,
             "t1": date_range[1]+time_root
            }

    # Check if all RGTs are considered, or only a subset
    if rgt != "all":
        parms["rgt"] = rgt
        print(f"Subsetting to only include ICESat-2 RGT {rgt}.")

    # Check for ATL08 filter
    if atl08_ids.get(atl08_class) != "None":
        parms["atl08_class"] = atl08_ids.get(atl08_class)
        print("Subsetting by selected ATL08 filter...")

    # Query SlideRule
    atl06sr = icesat2.atl06p(parms)

    return atl06sr

In [28]:
# Generate ICESat-2 data from SlideRule
atl06sr = atl06srq(field_id, date_range, rgt,
                 cnf_surface=cnf_surface,
                 atl08_class=atl08_class,
                 segment_length=segment_length,
                 res=res)

Subsetting to only include ICESat-2 RGT 152.
Subsetting by selected ATL08 filter...


In [29]:
# Convert ATL06SR to geodataframe in EPSG:32606
atl06sr['lon'], atl06sr['lat'] = atl06sr.geometry.x, atl06sr.geometry.y
atl06sr_gdf = atl06sr.to_crs('epsg:32606')

In [30]:
atl06sr

,rgt,gt,rms_misfit,pflags,spot,h_mean,cycle,region,dh_fit_dx,w_surface_window_final,n_fit_photons,h_sigma,segment_id,y_atc,x_atc,geometry,lon,lat
time,,,,,,,,,,,,,,,,,,
2023-03-31 08:03:54.178165248,152,10,0.152153,0,1,861.476163,19,5,0.020807,3.0,257,0.009513,619958,-20942.734375,12417928.0,POINT (-149.35693 68.64495),-149.356930,68.644950
2023-03-31 08:03:54.180987136,152,10,0.159902,0,1,861.835562,19,5,0.015121,3.0,261,0.009912,619959,-20942.583984,12417948.0,POINT (-149.35698 68.64477),-149.356981,68.644772
2023-03-31 08:03:54.183809024,152,10,0.190678,0,1,862.110205,19,5,0.013181,3.0,250,0.012092,619960,-20942.437500,12417968.0,POINT (-149.35703 68.64459),-149.357032,68.644593
2023-03-31 08:03:54.186630400,152,10,0.208933,0,1,862.384000,19,5,0.014664,3.0,267,0.012799,619961,-20942.298828,12417988.0,POINT (-149.35708 68.64441),-149.357083,68.644415
2023-03-31 08:03:54.189446912,152,10,0.219634,0,1,862.656698,19,5,0.011922,3.0,257,0.013713,619962,-20942.171875,12418008.0,POINT (-149.35713 68.64424),-149.357134,68.644237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 08:03:57.085337088,152,40,0.232235,0,4,807.166884,19,5,0.023660,3.0,21,0.050717,620871,-24282.724609,12436201.0,POINT (-149.48754 68.48549),-149.487536,68.485487
2023-03-31 08:03:57.088164608,152,40,0.176228,0,4,807.620950,19,5,0.017670,3.0,21,0.039386,620872,-24282.525391,12436221.0,POINT (-149.48758 68.48531),-149.487585,68.485308
2023-03-31 08:03:57.090991616,152,40,0.231219,0,4,807.756515,19,5,0.000944,3.0,20,0.052183,620873,-24282.359375,12436241.0,POINT (-149.48763 68.48513),-149.487634,68.485130


## Read Airborne Lidar Data

To derive snow depth with ICESat-2, we need a snow-off digital elevation model (DEM), which commonly originates from airborne lidar. This next step is designed to load and prepare some airborne lidar data from the University of Alaska, Fairbanks for this analysis.

In [32]:
import rioxarray as rio

# Snow-off lidar DEM
f_snow_off = f'{path}/lidar-dems/UKT_2023sept16_dtm_3m.tif'

# Snow-on lidar DEM (for comparison with the ICESat-2 results)
f_snow_on = f'{path}/lidar-dems/UKT_2023mar13_snowdepth_3m.tif'

# Read lidar DEMs into rioxarray format
lidar_snow_off = rio.open_rasterio(f_snow_off)
lidar_snow_on = rio.open_rasterio(f_snow_on)

## Co-Locate ICESat-2 and UAF Lidar

For this step, we will co-locate ICESat-2 and UAF so that we can directly compare the two datasets. The co-location will use a statistical method called "spline interpolation", and we will perform this co-location with both the snow-on and snow-off data.

The below function has the code needed to perform the co-location.

In [37]:
# Packages needed for the below functions
import geopandas as gpd
import numpy as np
import pandas as pd
from scipy.interpolate import RectBivariateSpline

def colocate_is2(lidar_snow_off, lidar_snow_on, is2_data):
    # Define x/y coordinates from snow-off data
    x0, y0 = np.array(lidar_snow_off.x), np.array(lidar_snow_off.y)

    # Do the same, but for the snow depth data
    xs, ys = np.array(lidar_snow_on.x), np.array(lidar_snow_on.y)

    # Remove filler values that would mess up the interpolator
    dem_heights = np.array(lidar_snow_off.sel(band=1))[::-1,:]
    dem_heights[np.isnan(dem_heights)] = -9999
    dem_depths = np.array(lidar_snow_on.sel(band=1))[::-1,:]
    dem_depths[np.isnan(dem_depths)] = -9999

    # Generate interpolator schemes
    interp_height = RectBivariateSpline(np.array(y0)[::-1], 
                                        np.array(x0),
                                        dem_heights)
    interp_depth = RectBivariateSpline(np.array(ys)[::-1],
                                       np.array(xs),
                                       dem_depths)

    # Use the spline interpolator to align the lidar with ICESat-2
    is2_lidar_df = pd.DataFrame()
    for beam in np.unique(is2_data['gt']):
        # Subset ICESat-2 data by current beam
        is2_tmp = is2_data.loc[is2_data['gt']==beam]

        # ICESat-2 x/y coordinates
        xn, yn = is2_tmp.geometry.x, is2_tmp.geometry.y

        # Define indices within x/y bounds of DEM
        i1 = (xn>np.min(x0)) & (xn<np.max(x0))
        i1 &= (yn>np.min(y0)) & (yn<np.max(y0))

        # Estimate lidar elevation and snow depth along ICESat-2 track
        lidar_height = interp_height(yn[i1], xn[i1], grid=False)
        lidar_depth = interp_depth(yn[i1], xn[i1], grid=False)

        # Construct dataframe of ICESat-2 and lidar data
        tmp = pd.DataFrame(data={'lat': is2_tmp['lat'][i1],
                                 'lon': is2_tmp['lon'][i1],
                                 'x': xn[i1],
                                 'y': yn[i1],
                                 'rgt': is2_tmp['rgt'][i1],
                                 'beam': is2_tmp['gt'][i1],
                                 'is2_height': is2_tmp['h_mean'][i1],
                                 'n_fit_photons': is2_tmp['n_fit_photons'][i1],
                                 'h_sigma': is2_tmp['h_sigma'][i1],
                                 'dh_fit_dx': is2_tmp['dh_fit_dx'][i1],
                                 'lidar_height': lidar_height,
                                 'lidar_snow_depth': lidar_depth
                                    }
                              )
        # Concatenate the co-located data into  final DataFrame
        is2_lidar_df = pd.concat([is2_lidar_df, tmp])

    return is2_lidar_df

In [ ]:
# Use the above function to co-locate the airborne lidar and ICESat-2
atl06sr_uaf = colocate_is2(lidar_snow_off, lidar_snow_on, atl06sr)

# Estimate the ICESat-2 snow depth
atl06sr_uaf['is2_snow_depth'] = atl06sr_uaf['is2_height'] - atl06sr_uaf['lidar_height']

# Convert final DataFrame in GeoDataFrame
atl06sr_uaf_gdf = gpd.GeoDataFrame(atl06sr_uaf,
                                   geometry=gpd.points_from_xy(atl06sr_uaf.lon, atl06sr_uaf.lat),
                                   crs='EPSG:4326')

atl06sr_uaf_gdf

An outline of the variables in our current GeoDataFrame:
* `lat` and `lon`: The latitude and longitude along the ICESat-2 track.
* `x` and `y`: The easting and northing along the ICESat-2 track, in projection EPSG:32606.
* `rgt`: The reference ground track number of the ICESat-2 track of interest.
* `beam`: The ICESat-2 beam designation (gt1l, gt2r, etc.)
* `is2_height`: ICESat-2 height estimate at the given location.
* `n_fit_photons`: Number of ICESat-2 photons used to derive `is2_height`.
* `h_sigma`: Approximate uncertainty of `is2_height`.
* `dh_fit_dx`: A rough measure of surface slope along the ICESat-2 track.
* `lidar_height`: Lidar height estimate at the given location.
* `lidar_snow_depth`: Lidar snow depth estimate at the given location.
* `is2_snow_depth`: ICESat-2 snow depth estimate at the given location.

The key variables are `is2_snow_depth` and `lidar_snow_depth` for our comparisons. Several of the other variables, such as `n_fit_photons` and `h_sigma`, can be used to filter or process the data further, if desired.

Let's look at a simple comparison between the two depth products.

In [ ]:
import matplotlib.pyplot as plt

# Line plot of along-track snow depths
fig, ax = plt.subplots(figsize=(9,6))
atl06sr_uaf_gdf.plot(kind='scatter', ax=ax, x='lat', y='is2_snow_depth',
                     linewidth=3, label='ICESat-2')
atl06sr_uaf_gdf.plot(kind='scatter', ax=ax, x='lat', y='lidar_snow_depth',
                     linewidth=1.5, label='UAF lidar')
ax.set_xlabel('Latitude', fontsize=14)
ax.set_ylabel('Snow depth [m]', fontsize=14)
ax.set_ylim([0, 1.5])
ax.legend()

We can also calculate the difference in snow depth between ICESat-2 and UAF, then make a spatial plot using `geopandas.explore()`.

In [ ]:
# Calculate snow depth bias
atl06sr_uaf_gdf['snow_depth_residual'] = atl06sr_uaf_gdf['is2_snow_depth'] - atl06sr_uaf_gdf['lidar_snow_depth']

# Create a spatial plot of the snow depth bias
atl06sr_uaf_gdf.explore(column='snow_depth_residual', 
                        tiles='Esri.WorldImagery',
                        cmap='viridis',
                        vmin=-1.5, vmax=1.5)

If the data looks good, then we can save the final GeoDataFrame as a geoJSON.

In [ ]:
# Save the GeoDataFrame
atl06sr_uaf_gdf.to_file(f'{path}/is2_uaf_snow-depths.geojson',
                        driver='GeoJSON')